## Load Data and Tokenizer

In [ ]:
from transformers import BertTokenizer
from datasets import load_dataset

ds = load_dataset('billingsmoore/tagged-tibetan-to-english-translation-dataset')

tokenizer = BertTokenizer.from_pretrained('tibetan_tokenizer')

## Preprocess Data

### Use just first two tags

In [ ]:
def just_two_tags(examples):
    tags = [tag[:2] for tag in examples['Tags']]
    examples['Tags'] = tags
    return examples

ds = ds.map(just_two_tags, batched=True)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer()
labels = mlb.fit(ds['train']['Tags'])  # Fit all unique Tags

# Save label mappings
import json
with open("label_mapping.json", "w") as f:
    json.dump(mlb.classes_.tolist(), f)


In [ ]:
def preprocess(examples):
    tokens = tokenizer(examples["Tibetan"], padding="max_length", truncation=True, max_length=128)
    tokens["labels"] =  mlb.transform(examples['Tags']).astype(float).tolist() # Convert labels to multi-hot
    return tokens

encoded_dataset = ds.map(preprocess, batched=True)


In [ ]:
encoded_dataset = encoded_dataset.remove_columns(['Tibetan', 'Phonetic', 'English', 'Tags'])

In [ ]:
encoded_dataset = encoded_dataset['train'].train_test_split(.15)

## Train Model

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load tokenizer and model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(mlb.classes_))

# Resize embeddings to match the new tokenizer
model.resize_token_embeddings(len(tokenizer))

# Move model to GPU
model = model.to('cuda:0')

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(eval_pred):
    predictions, references = eval_pred
    
    # Apply a threshold to convert logits to binary predictions
    predictions = (predictions > 0.5).astype(int)
    
    # Compute metrics
    accuracy = accuracy_score(references, predictions)
    f1 = f1_score(references, predictions, average="micro")
    precision = precision_score(references, predictions, average="micro")
    recall = recall_score(references, predictions, average="micro")
    
    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

from transformers import EarlyStoppingCallback

# Define training arguments
training_args = TrainingArguments(
    output_dir="bert-classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,  # Set a maximum number of epochs
    weight_decay=0.01,
    eval_strategy="epoch",  # Evaluate at the end of every epoch
    save_strategy="epoch",  # Save the model at the end of every epoch
    load_best_model_at_end=True,  # Load the best model after training
    metric_for_best_model="accuracy",  # Metric to monitor
    greater_is_better=True,  # Higher accuracy is better
    logging_dir="./logs"
)

# Add the EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3  # Stop training if the metric does not improve for 3 evaluation steps
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]  # Add the early stopping callback
)

# Start training
trainer.train()